In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyswarms as ps
# from pyswarms.utils.functions import single_obj as fx
# from pyswarms.utils.plotters import plot_cost_history
from IPython.display import display

## Load previous results

### load price elasticity

In [ ]:
categories_enc =  {'Aquatic Roots and Tubers':'1', 'Cauliflower':'2' ,'Chili Peppers':'3', 'Edible Mushrooms':'4', 'Leafy Greens':'5', 'Solanaceous Vegetables':'6'}

price_elasticity = []

for category in categories_enc.keys():
    category = category.replace(" ", "_")
    coef_df = pd.read_csv(f'../results/reg_coef_{category}.csv',index_col=0)

    paddings_ = list(set(range(1,7)) - set([int(idx.split('_')[-1]) for idx in coef_df.index if 'avg_sale_price' in idx]))
    for padding in paddings_:
        coef_df.loc[f'avg_sale_price_{padding}','coef'] = 0
    coef_df = coef_df.loc[[idx for idx in coef_df.index if 'avg_sale_price' in idx],['coef']]
    coef_df.sort_index(inplace=True)
    price_elasticity.append(coef_df['coef'].values)

price_elasticity = np.array(price_elasticity)
print(price_elasticity.shape)
# price_elasticity[i][j] =  the demand elasticity of category i to price of category j

### load prediction of sale volume, wholesale price

In [ ]:
pred_sale_volume = np.load('../results/pred_sale_volume.npy')
pred_wholesale_price = np.load('../results/pred_wholesale_price.npy')
print(pred_sale_volume.shape, pred_wholesale_price.shape)
# pred_XX[i][j] = the predicted sale volume/wholesale price of day i of category j

In [ ]:
daily_category_sales = pd.read_csv('../data/daily_category_sales.csv',index_col=0, header=[0,1])
recent_sales = daily_category_sales.iloc[-7:,:]
recent_prices = (recent_sales['cost_sum'] + recent_sales['sales_profit_sum']) / recent_sales['quantity_sum']
avg_recent_prices = recent_prices.mean().values

discounts = [0.7]*6
spoilage_rates = [0.1365, 0.1551, 0.0924, 0.0945, 0.1283, 0.0668]

### display profit and profit rate of last week

In [ ]:
display(recent_sales['sales_profit_sum'].sum(axis=1))
display(recent_sales['avg_profit_rate'])

## Set up rules

### Adjust demand volume with estimated Price elasticity of demand

$$
\Delta \ln Q_{k} = \sum\limits_{i=1}^m \beta_{ki} \Delta \ln P_{i} \ ,\hspace{10pt} \text{i.e.} \ \ln \frac{Q_k}{Q_{base,k}} = \sum\limits_{i=1}^m \beta_{ki} \ln \frac{P_{i}}{P_{base,i}} \hspace{5pt} (k=1,2,\cdots,m)
$$
where $Q_{base}$ is the predict value of LSTM, and $P_{base}$ is last week's average selling price of the category

In [ ]:
def adjust_demand(q_base, p_curr, p_base=avg_recent_prices, p_elasticity=price_elasticity):
    """
    p_base: the base price
    p_curr: the current price
    q_base: the base demand
    p_elasticity: the price elasticity
    """
    q_log_delta = np.matmul(p_elasticity, np.log(p_curr/p_base))

    return q_base * np.exp(q_log_delta)

### Calculate daily profit

In [ ]:
def daily_profit(profit_rate, q_buyin, 
                pred_sale_volume=pred_sale_volume, 
                pred_wholesale_price=pred_wholesale_price, 
                discounts=discounts, 
                spoilage_rates=spoilage_rates):

    p_buyin = pred_wholesale_price[day] # The external variable day controls the day to forecast
    q_demand_pred = pred_sale_volume[day]

    p_sale = np.multiply(p_buyin, np.ones(6)+profit_rate)
    p_sale_discount = np.multiply(p_sale, discounts)
    q_demand_adjusted = adjust_demand(q_base=q_demand_pred, p_curr=p_sale)

    q_buyin_normal = np.multiply(q_buyin, (np.array([1]*6)-spoilage_rates))
    q_buyin_discount = np.multiply(q_buyin, spoilage_rates)

    profit = 0
    for idx in range(6):
        cost = p_buyin[idx]*q_buyin[idx]
        if q_buyin[idx] <= q_demand_adjusted[idx]:
            profit += (p_sale[idx]*q_buyin_normal[idx] + p_sale_discount[idx]*q_buyin_discount[idx]) - cost
        else:
            profit += p_sale[idx]*np.multiply(q_demand_adjusted, (np.array([1]*6)-spoilage_rates))[idx] + \
                p_sale_discount[idx]*np.multiply(q_demand_adjusted, spoilage_rates)[idx] - cost
        
    return profit

In [ ]:
def objective_func(X): # X: (n_particles, 12) = (particles, concat(profit_rate, buyin_quantity))
    profit_list = []
    
    for i in range(n_particles):
        particle_pos = X[i]

        profit_rate = particle_pos[:6]
        q_buyin = particle_pos[6:]
        profit = daily_profit(profit_rate, q_buyin)
        profit_list.append(-profit)
    
    return profit_list

## Optimize profit

### Initailize particle positions

In [ ]:
# use average value of recent 30 days for initialization
print(daily_category_sales['quantity_sum'].iloc[-30:].mean())
print(daily_category_sales['avg_profit_rate'].iloc[-30:].mean())

In [ ]:
ini_pos = np.array([0.4, 0.5, 0.7, 0.5, 0.5, 0.6] + [20, 20, 90, 60, 150, 30]) # initialize wholesale prices and sale volume

n_particles = 1000
n_dimensions = 12
lower_bound = np.array([0.1]*6 + [1]*6)
upper_bound = np.array([1]*6 + [100,100,250,250,450,100])
bounds = (lower_bound, upper_bound)

# some particles have the given initial position, while others have random positions
weight_ini = 0.3 
pos_given = np.random.uniform(low=lower_bound, high=upper_bound, size=(int(n_particles*weight_ini), n_dimensions))
pos_given = 0.2 * pos_given + 0.8 * ini_pos
pos_random = np.random.uniform(low=lower_bound, high=upper_bound, size=(int(n_particles*(1-weight_ini)), n_dimensions))
initial_pos = np.vstack((pos_given, pos_random))

In [ ]:
optim_result = pd.DataFrame(columns=['date', 'profit_rate_1', 'profit_rate_2', 'profit_rate_3', 'profit_rate_4', 'profit_rate_5','profit_rate_6',
                            'q_buyin_1', 'q_buyin_2', 'q_buyin_3', 'q_buyin_4','q_buyin_5', 'q_buyin_6',
                            'max_profit_pred'])
optim_history = []
for day in range(7):    
    options = {'c1': 0.7, 'c2': 0.5, 'w': 0.9} # hyperparameters of PSO
    optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, 
                                        dimensions=n_dimensions, 
                                        options=options, 
                                        bounds=bounds,
                                        init_pos=initial_pos,
                                        ftol_iter=100,
                                        ftol=1e-4,
                                        oh_strategy={ "w":'exp_decay', "c1":'nonlin_mod',"c2":'lin_variation'}
                                        )

    best_position, best_cost = optimizer.optimize(objective_func, iters=1000, verbose=True)

    i = optim_result.shape[0]
    optim_result.loc[i, 'date'] = pd.to_datetime('2023-07-01') + pd.to_timedelta(i, unit='d')
    optim_result.iloc[i,1:-1] = best_cost
    optim_result.iloc[i,-1] = -best_position

    optim_history.append(optimizer.cost_history)

optim_result['date'] = pd.to_datetime(optim_result['date'])


In [ ]:
sns.set_theme(style="darkgrid")
fig, ax = plt.subplots(2,4,figsize=(24,10), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.4)

for day in range(7):
    profits = np.array(optim_history[day])*-1
    i, j = (day)//4, (day)%4
    sns.lineplot(data=profits, ax=ax[i][j])
    ax[i][j].set_title(f"Day {day+1}")
    ax[i][j].set_xlabel("Iteration")
    ax[i][j].set_ylabel("Profit")

In [ ]:
optim_result.to_csv('../results/max_profit_results.csv',index=False)

## Post-process the optimization results to fit the reality

In [ ]:
raw_res = pd.read_csv('../results/max_profit_results.csv',index_col=0)
raw_res.head(10)

In [ ]:
# round sale price to 2 decimal places (0.01 CNY/kg), round wholesale quantity to 1 decimal place (0.1kg)
profit_rates = raw_res[[f'profit_rate_{i+1}' for i in range(6)]].values
q_buyin = raw_res[[f'q_buyin_{i+1}' for i in range(6)]].values
practical_sale_price = np.round((profit_rates + np.ones((7,6))) * pred_wholesale_price, 2)
practical_q_buyin = np.round(q_buyin, 1)

In [ ]:
# re-calculate the expected profit
expected_profit = []
for d in range(7):
    q_normal =  practical_q_buyin[d]* (np.ones(6)-spoilage_rates)
    q_discount = practical_q_buyin[d]* spoilage_rates
    income = np.dot(practical_sale_price[d], q_normal) + np.dot(practical_sale_price[d]*discounts, q_discount)
    cost = np.dot(pred_wholesale_price[d], practical_q_buyin[d])
    profit = income - cost
    expected_profit.append(profit)
    
expected_profit = np.round(expected_profit, 2)

In [ ]:
processed_res = pd.DataFrame(columns=[f'sale_price_{list(categories_enc.keys())[i]}'.replace(' ','_') for i in range(6)] + [f'q_buyin_{list(categories_enc.keys())[i]}' for i in range(6)] + ['max_profit_pred'], index=raw_res.index)

processed_res.iloc[:,:6]=practical_sale_price
processed_res.iloc[:,6:12]=practical_q_buyin
processed_res.iloc[:,-1]=expected_profit

processed_res.head(10)

In [ ]:
processed_res.to_excel('../results/strategy_and_profit_final.xlsx')